<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/vit/paligemma/modelling_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! pip install import-ipynb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00


In [17]:
!ls drive/MyDrive/Colab\ Notebooks/PyTorch\ Transformer/Vision\ Transformer/paligemma

modelling_gemma.ipynb  processing_paligemma.ipynb  siglip.ipynb


In [23]:
import sys
import os
sys.path.append("drive/MyDrive/Colab\ Notebooks/PyTorch\ Transformer/Vision\ Transformer/paligemma")
os.cwd("drive/MyDrive/Colab\ Notebooks/PyTorch\ Transformer/Vision\ Transformer/paligemma")

AttributeError: module 'os' has no attribute 'cwd'

In [21]:
import import_ipynb
from siglip import SiglipVisionConfig

ModuleNotFoundError: No module named 'siglip'

In [2]:
import torch
from torch import nn
from typing import Optional, Tuple, List
from torch.nn import CrossEntropyLoss
import math

In [ ]:
class GemmaConfig:

  def __init__(
      self,
      vocab_size,
      hidden_size,
      intermediate_size,
      num_hidden_layers,
      num_attention_heads,
      num_key_value_heads,
      head_dim=256,
      max_position_embedding=8192,
      rms_norm_eps=1e-6,
      rope_theta=10000.0,
      attention_bias=False,
      attention_dropout=0.0,
      pad_token_id=None,
      **kwargs
  ):

    super().__init__()
    self.vocab_size = vocab_size
    self.max_position_embedding = max_position_embedding
    self.hidden_size = hidden_size
    self.intermediate_size = intermediate_size
    self.num_hidden_layers = num_hidden_layers
    self.num_attention_heads = num_attention_heads
    self.head_dim = head_dim
    self.num_key_value_heads = num_key_value_heads
    self.rms_norm_eps = rms_norm_eps
    self.rope_theta = rope_theta
    self.attention_bias = attention_bias
    self.attention_dropout = attention_dropout
    self.pad_token_id = pad_token_id

In [4]:
class PaliGemmaConfig:

  def __init__(
      self,
      vision_config=None,
      text_config=None,
      ignore_index=-100,
      image_token_index=256000,
      vocab_size=257152,
      projection_dim=2048,
      hidden_size=2048,
      pad_token_id=None,
      **kwargs
  ):
    super().__init__()
    self.ignore_index = ignore_index
    self.image_token_index = image_token_index
    self.vocab_size = vocab_size
    self.projection_dim = projection_dim
    self.hidden_size = hidden_size
    self.vision_config = vision_config
    self.is_encoder_decoder = False,
    self.pad_token_id = pad_token_id

    self.vision_config = SiglipVisionConfig(**vision_config)
    self.text_config = text_config
    self.text_config = GemmaConfig(**text_config, pad_token_id=pad_token_id)
    self.vocab_size = self.text_config.vocab_size
    self.text_config.num_image_tokens = (self.vision_config.image_size // self.vision_config.patch_size) ** 2
    self.vision_config.projection_dim = projection_dim

In [ ]:
class GemmaModel(nn.Module):

  def __init__(self, config: GemmaConfig):
    super().__init__()
    self.config = config
    self.padding_idx = config.pad_token_id
    self.vocab_size = config.vocab_size

    self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size, self.padding_idx)
    self.layers = nn.ModuleList(
        [GemmaDecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]
    )
    self.norm = GemmaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

  def get_input_embeddings(self):
    return self.embed_tokens

  def forward(
      self,
      attention_mask: Optional[torch.Tensor] = None,
      position_ids: Optional[torch.LongTensor] = None,
      inputs_embeds: Optional[torch.FloatTensor] = None,
      kv_cache: Optional[KVCache] = None
  ) -> torch.FloatTensor:
    hidden_states = inputs_embeds # (B, seq_len, hidden_size)
    normalizer = torch.tensor(self.config.hidden_size**0.5, dtype=hidden_states.dtype)
    hidden_states = hidden_states * normalizer

    for decoder_layer in self.layers:
      hidden_states = decoder_layer( # (B, seq_len, hidden_size)
          hidden_states,
          attention_mask=attention_mask,
          position_ids=position_ids,
          kv_cache=kv_cache
      )

    hidden_states = self.norm(hidden_states) # (B, seq_len, hidden_size)

    return hidden_states

In [ ]:
class GemmaForCausalLM(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.config = config
    self.model = GemmaModel(config)
    self.vocab_size = config.vocab_size
    self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

  def get_input_embeddings(self):
    return self.model.embed_tokens

  def tie_weights(self):
    self.lm_head.weight = self.model.embed_tokens.weight

  def forward(
      self,
      attention_mask: Optional[torch.Tensor] = None,
      position_ids: Optional[torch.LongTensor] = None,
      inputs_embeds: Optional[torch.FloatTensor] = None,
      kv_cache: Optional[KVCache] = None
  ) -> Tuple:
    outputs = self.model(
        attention_mask=attention_maks,
        position_ids=position_ids,
        inputs_embeds=inputs_embeds,
        kv_cache=kv_cache,
    )

    hidden_states = outputs
    logits = self.lm_head(hidden_states)
    logits = logits.float()

    return_data = {
        "logits": logits,
    }

    if kv_cache is not None:
      # Return the updated cache
      return_data["kv_cache"] = kv_cache

    return kv_cache

In [ ]:
class PaliGemmaMultiModalProjector(nn.Module):

  def __init__(self, config: PaliGemmaConfig):
    super().__init__()
    self.linear = nn.Linear(config.vision_config.hidden_size, config.vision_config.projection_dim, bias=True)

  def forward(self, image_features):
    hidden_states = self.linear(image_features) # (B, num_patches, projection_dim)
    return hidden_states

In [2]:
class PaliGemmaForConditionalGeneration(nn.Module):

  def __init__(self, config: PaliGemmaConfig):
    super().__init__()
    self.config = config
    self.vision_tower = SiglipVisionModel(config.vision_config)
    self.multi_modal_projector = PaliGemmaMultiModalProjector(config)
    self.vocab_size = config.vocab_size

    language_model = GemmaForCausalLM(config.text_config)
    self.language_model = language_model
    self.pad_token_id = self.config.pad_token_id if self.config.pad_token_id is not None else -1

  def tie_weights(self):
    return self.language_model.tie_weights()

  def _merge_input_ids_with_image_features(
      self, image_features: torch.Tensor,
      inputs_embeds: torch.Tensor,
      input_ids: torch.Tensor,
      attention_mask: torch.Tensor,
      kv_cache: KVCache
  ):
    _, _, embed_dim = image_features.shape
    batch_size, sequence_length = input_ids.shape
    dtype, device = inputs_embeds.dtype, inputs_embeds.device
    scaled_image_features = image_features / (self.config.hidden_size**0.5) # (B, seq_len, hidden_size)

    # combine the embeddings of the image tokens, the text tokens and mask out all the padding tokens
    final_embedding = torch.zeros(batch_size, sequence_length, embed_dim, dtype=inputs_embeds.dtype, device=inputs_embeds.device)
    text_mask = (input_ids != self.config.image_token_index) & (input_ids != self.pad_token_id) # (B, seq_len)
    image_mask = input_ids == self.config.image_token_index # (B, seq_len)
    pad_mask = input_ids == self.pad_token_id # (B, seq_len)

    # We need to expand the masks to the embedding dimension otherwise we can't use them in torch.where
    text_mask_expanded = text_mask.unsqueeze(-1).expand(-1, -1, embed_dim)
    pad_mask_expanded = pad_mask.unsqueeze(-1).expand(-1, -1, embed_dim)
    image_mask_expanded = image.unsqueeze(-1).expand(-1, -1, embed_dim)

    # add the text embedding
    final_embedding = torch.where(text_mask_expanded, inputs_embeds, final_embedding)
    # insert image embeddings. We can't use torch.where because sequence length of scaled_image_features is different from final_embedding
    final_embedding = torch.masked_scatter(image_mask_expanded, scaled_image_features)
    # Zero out padding tokens
    final_embedding = torch.where(pad_mask_expanded, toch.zeros_like(final_embedding), final_embedding)

    ### CREATE THE ATTENTION MASK ###
    dtype, device = inputs_embeds.dtype, inputs_embeds.device
    min_dtype = torch.finfo(dtype).min
    q_len = inputs_embeds.shape[1]

    if kv_cache is None or kv_cache.num_items() == 0:
      # Do not mask any token, because we're in the prefill phase
      # This only works when we have no padding
      causal_mask = torch.full(
          (batch_size, q_len, q_len), fill_value=0, dtype=dtype, device=device
      )
    else:
      # Since we are generating tokens, the query must be one single token
      assert q_len == 1
      kv_len = kv_cache.num_items() + q_len
      # Also in this case we don't need to mask anything, since each query shoud be able to attend all previous tokens
      # this only works when we have no padding
      causal_mask = torch.full(
          (batch_size, q_len, kv_len), fill_value=0, dtype=dtype, device=device
      )

      # Add the head dimension
      causal_mask = causal_mask.unsqueeze(1) # (B, num_heads_q, q_len, kv_len)

      if kv_cache is not None and kv_cache.num_items() > 0:
        # the position of the query is just the last position
        position_ids = attention_mask.cumsum(-1)[:, -1]
        if position_ids.dim() == 1:
          position_ids = position_ids.unsqueeze(0)

      else:
        # Create a position_ids based on the size of the attention_mask
        # For masked tokens, use the number 1 as position
        position_ids = (attention_mask.cumsum(-1)).masked_fill_((attention_mask == 0), 1).to(device)

    return final_embedding, causal_mask, position_ids


  def forward(
      self,
      input_ids: torch.LongTensor = None,
      pixel_values: torch.FloatTensor = None,
      attention_mask: Optional[torch.Tensor] = None,
      kv_cache: Optional[KVCache] = None,
  ) -> Tuple:
    assert torch.all(attention_mask == 1), "The input cannot be padded"
    inputs_embeds = self.language_model.get_input_embeddings()(input_ids) # (B, seq_len, hidden_size)
    selected_image_feature = self.vision_tower(pixel_values.to(inputs_embeds.dtype)) # (B, num_patches, embed_dim)
    image_features = self.multi_modal_projector(selected_image_feature) # (B, num_patches, hidden_size)
    # Merge the embeddings of the text tokens and the image tokens
    input_embeds, attention_mask, position_ids = self._merge_input_ids_with_image_features(image_features, inputs_embeds, input_ids, attention_mask, kv_cache)

    outputs = self.language_model(
        attention_mask=attention_mask,
        position_ids=position_ids,
        inputs_embeds=inputs_embeds,
        kv_cache=kv_cache
    )

    return outputs

NameError: name 'nn' is not defined

In [ ]:
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
